In [15]:
from webapp import create_app
from vk_parser import *
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
from time import time
%matplotlib inline

# Обработка текста.

- Обработка текста через регулярные выражения.
- Удаление всех url из текста.
- Удаление хэштегов, если они есть в тексте.
- Удаление имён пользователей.
- Удаление пунктуации и цифр.
- Приведение всего текста к нижнему регистру.

In [26]:
def preprocess_text(text):
    text = text.lower().replace('ё', 'е')
    text = text.replace('d', '')
    text = text.replace('rt', '')
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'\b[a-zA-Zа-яА-Я1-9]\b', '', text)
    text = re.sub(r'\b(?:[аевзпхщчоумы][аиеоузпхщчмы]+)+\b', '', text)
    text = re.sub(r'[0-9]', '', text)    
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

# Считаем частоту слов в нашем предобработанном датасете

In [44]:
wordfreq = {}

for token in default_list:
    if token not in wordfreq.keys():
        wordfreq[token] = 1
    else:
        wordfreq[token] += 1

In [12]:
#wordfreq

In [19]:
res = np.array(default_list)

In [20]:
unique_res = np.unique(res) 
unique_res

array(['ааа', 'ааааа', 'абаддон', ..., 'ярость', 'ясно', 'ёлочка'],
      dtype='<U19')

In [21]:
len(unique_res)

3507

In [22]:
bd_unique_words = unique_res.tolist()

# Предобработка текста.

- Используем корпус Юлии Рубцовой русских твитов для тренировки модели.
- Скачиваем корпус позитивных и негативных твитов с сайта http://study.mokoron.com/
- Добавляем дополнительную колонку для позитивных (1) и негативных твитов (0), тем самым размечая данные.

In [27]:
# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('training_data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('training_data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])

raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [28]:
app = create_app()
with app.app_context():
    query = Comment.query.all()
    preprop_comm = []

    for comment in query:
        preprop_comm.append(preprocess_text(comment.comment_text))

In [29]:
data = [preprocess_text(t) for t in raw_data]

In [30]:
bd_data = [preprocess_text(t) for t in preprop_comm]

In [31]:
print(len(labels), len(raw_data), len(data))

223846 223846 223846


In [32]:
print(len(bd_data))

1220


- Скачиваем набор русских стоп-слов для предобработки наших твитов.

In [33]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

- Разбиваем предобработанные слова на токены.
- Удаляем стоп слова.
- Производим лемматизацию или стемминг текста.
- Удаляем короткие слова длинной меньше 2 символов.

In [34]:
def tokenaizer(data):
    tokens= [word_tokenize(sent) for sent in data]
    return tokens

In [35]:
tokenized_data = tokenaizer(data)

In [36]:
bd_tokenized_data = tokenaizer(bd_data)

In [37]:
print(len(bd_tokenized_data))

1220


In [38]:
print(len(tokenized_data))

223846


# Стемминг слов

In [39]:
snowball = SnowballStemmer(language="russian")
def stemmer(word, stem_init):
    return stem_init.stem(str(word))

In [40]:
stop_words = set(stopwords.words("russian"))
def delete_stop_words_with_stemming(words, stop_words):    
    default_list = []
    for word in words:
        if word not in stop_words:
            stemmed_word = stemmer(word, snowball)
            default_list.append(stemmed_word)

    return list(default_list)

In [39]:
stemmed_words = []
counter = 0
for part_of_list in tokenized_data:
    stemmed_words.append(delete_stop_words_with_stemming(part_of_list, stop_words))
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 5
Processed 3000 of 15
Processed 4000 of 8
Processed 5000 of 13
Processed 6000 of 13
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 18
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 5
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 9
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 10
Processed 22000 of 2
Processed 23000 of 6
Processed 24000 of 11
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 4
Processed 29000 of 13
Processed 30000 of 9
Processed 31000 of 14
Processed 32000 of 3
Processed 33000 of 4
Processed 34000 of 6
Processed 35000 of 16
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 15
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 5
Processed 45000 of 9
Processed 46000 of 12
Processed 47000 of 9
Proc

In [40]:
print(len(stemmed_words))

223846


# Лемматизация слов.

"Экземпляры класса MorphAnalyzer обычно занимают порядка 15Мб оперативной памяти (т.к. загружают в память словари, данные для предсказателя и т.д.); старайтесь организовать свой код так, чтобы создавать экземпляр MorphAnalyzer заранее и работать с этим единственным экземпляром в дальнейшем."

https://pymorphy2.readthedocs.io/en/latest/user/guide.html

In [41]:
morph = pymorphy2.MorphAnalyzer()
def delete_stop_words_with_lemmatizing(words, morph, stop_words):  
    default_list = []
    for word in words:
        if word not in stop_words:
            lem_word = morph.parse(str(word))[0].normal_form
            default_list.append(lem_word)
    return list(default_list)

In [42]:
lemmatized_words = []
counter = 0
for part_of_list in tokenized_data:
    lemmatized_words.append(delete_stop_words_with_lemmatizing(part_of_list, morph, stop_words))   
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 5
Processed 3000 of 15
Processed 4000 of 8
Processed 5000 of 13
Processed 6000 of 13
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 18
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 5
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 9
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 10
Processed 22000 of 2
Processed 23000 of 6
Processed 24000 of 11
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 4
Processed 29000 of 13
Processed 30000 of 9
Processed 31000 of 14
Processed 32000 of 3
Processed 33000 of 4
Processed 34000 of 6
Processed 35000 of 16
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 15
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 5
Processed 45000 of 9
Processed 46000 of 12
Processed 47000 of 9
Proc

In [23]:
len(lemmatized_words)

223846

In [45]:
lemmatized_words

[['школотый',
  'поверь',
  'самый',
  'общество',
  'профилировать',
  'предмет',
  'тип'],
 ['таки', 'немного', 'похожий', 'мальчик', 'равно'],
 ['идиотка', 'испугаться'],
 ['угол',
  'сидеть',
  'погибать',
  'голод',
  'порция',
  'взять',
  'хотя',
  'жрать',
  'хотеть'],
 ['значит',
  'страшилка',
  'блин',
  'посмотреть',
  'часть',
  'создаться',
  'ощущение',
  'автор',
  'курить'],
 ['любить', 'знать', 'бля'],
 ['это', 'друг', 'такой'],
 ['ухажёр', 'ухо', 'жрать'],
 ['поприветствовать', 'мой', 'новый', 'читатель'],
 ['частичка', 'сидней'],
 ['любить', 'остальной', 'влюбляться', 'привязываться'],
 ['забыть',
  'вчера',
  'приехать',
  'деревня',
  'наш',
  'котэнить',
  'испугаться',
  'утром',
  'прийти',
  'к',
  'общаться',
  'добрый'],
 ['задумать'],
 ['дааааа'],
 ['равно',
  'обзор',
  'урезать',
  'просто',
  'один',
  'версия',
  'сильно',
  'порезать',
  'другой',
  'полный',
  'мусор'],
 ['смотреть',
  'случайно',
  'аспирантура',
  'попасть',
  'наука',
  'никто',
  

In [32]:
bd_lemmatized_words = []
counter = 0
for part_of_list in bd_tokenized_data:
    bd_lemmatized_words.append(delete_stop_words_with_lemmatizing(part_of_list, morph, stop_words))   
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 2


In [33]:
len(bd_lemmatized_words)

1220

In [34]:
bd_lemmatized_words = [x for x in bd_lemmatized_words if x != []] #удалил пустые списки после лемматизации

In [35]:
len(bd_lemmatized_words)

1070

In [87]:
def to_data_frame(data, labels):
    default_list = []
    
    
    for string in data:
        new_string = ' '.join(string)
        default_list.append(new_string)
    
    
    default_list = pd.DataFrame(list(zip(default_list, labels)), columns =['comments', 'sentiment'])
    return default_list

In [42]:
stemmed_words = to_data_frame(stemmed_words, labels)

In [88]:
lemmatized_words = to_data_frame(lemmatized_words, labels)

In [89]:
lemmatized_words

,comments,sentiment
0,школотый поверь самый общество профилировать п...,1
1,таки немного похожий мальчик равно,1
2,идиотка испугаться,1
3,угол сидеть погибать голод порция взять хотя ж...,1
4,значит страшилка блин посмотреть часть создать...,1
...,...,...
223841,каждый хотеть исправлять,0
223842,скучать вправлять мозг равно скучать,0
223843,школа говно это идти,0
223844,тауриэль грусть обнять,0


- сохраним наш датасет

In [34]:
lemmatized_words.to_csv('lemmatized_words.csv', index=False, encoding='cp1251')

In [44]:
stemmed_words.to_csv('stemmed_words.csv', index=False, encoding='cp1251')

In [39]:
X_test.to_csv('X_test.csv', index=False, encoding='cp1251')

- загрузим наш датасет

In [99]:
lemmatized_words = pd.read_csv('lemmatized_words.csv', encoding='cp1251')

In [18]:
stemmed_words = pd.read_csv('stemmed_words.csv', encoding='cp1251')

In [19]:
bd_lemmatized_words = pd.read_csv('X_test.csv', encoding='cp1251')

In [107]:
lemmatized_words

,comments,sentiment
0,школотый поверь самый общество профилировать п...,1
1,таки немного похожий мальчик равно,1
2,идиотка испугаться,1
3,угол сидеть погибать голод порция взять хотя ж...,1
4,значит страшилка блин посмотреть часть создать...,1
...,...,...
223841,каждый хотеть исправлять,0
223842,скучать вправлять мозг равно скучать,0
223843,школа говно это идти,0
223844,тауриэль грусть обнять,0


# Word embedding (Векторное представление слов)

- Сделаем векторизацию текста 4-мя способами BoW, TF-IDF, Word2Vec, doc2vec.
- На этих данных обучим модели и сравним результаты.

# Bag Of Words (BoW).

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lemmatized_words['comments'])

In [68]:
print(vectorizer.get_feature_names()[:100])

['аа', 'ааа', 'аааа', 'ааааа', 'аааааа', 'ааааааа', 'аааааааа', 'ааааааааа', 'аааааааааа', 'ааааааааааа', 'аааааааааааа', 'ааааааааааааа', 'аааааааааааааа', 'ааааааааааааааа', 'аааааааааааааааа', 'ааааааааааааааааа', 'аааааааааааааааааа', 'ааааааааааааааааааа', 'аааааааааааааааааааа', 'ааааааааааааааааааааа', 'аааааааааааааааааааааа', 'ааааааааааааааааааааааа', 'аааааааааааааааааааааааа', 'ааааааааааааааааааааааааа', 'аааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'ааааааааааааааааааа

In [39]:
print(X.shape)

(223846, 94121)


# Bag Of Word with n-grams.

In [64]:
vectorizer2 = CountVectorizer(ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(lemmatized_words['comments'])

In [65]:
print(vectorizer2.get_feature_names()[:100])

['аа аа', 'аа ааа', 'аа айн', 'аа акаарыбын', 'аа ап', 'аа арай', 'аа ардчилал', 'аа ахи', 'аа бантик', 'аа бедный', 'аа бифри', 'аа бла', 'аа блин', 'аа бля', 'аа блянулааан', 'аа бог', 'аа болеть', 'аа болзоон', 'аа брат', 'аа быть', 'аа верить', 'аа вон', 'аа вообще', 'аа воот', 'аа вспомнить', 'аа вчера', 'аа вще', 'аа выспаться', 'аа выходить', 'аа говорить', 'аа год', 'аа горло', 'аа господь', 'аа гэж', 'аа даа', 'аа давать', 'аа делать', 'аа день', 'аа дианыч', 'аа долго', 'аа думать', 'аа жарко', 'аа ждать', 'аа заболеть', 'аа забыть', 'аа завтра', 'аа заебалиия', 'аа закончиться', 'аа замёрзлый', 'аа запутаться', 'аа зараз', 'аа заставить', 'аа зато', 'аа збс', 'аа значит', 'аа извинить', 'аа ита', 'аа каждый', 'аа казаться', 'аа капец', 'аа каток', 'аа киев', 'аа класс', 'аа ключ', 'аа код', 'аа красота', 'аа крилла', 'аа ладный', 'аа лан', 'аа любить', 'аа маленький', 'аа мочь', 'аа наверно', 'аа нада', 'аа наивный', 'аа народ', 'аа начать', 'аа начаться', 'аа недавно', 'аа 

In [66]:
print(X2.shape)

(223846, 781179)


- Вывод: 
  - Потеря порядка слов;
  - Полученные вектора, не являются информативными и занимают очень много места в памяти;

# TF-IDF.

(TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(data,tfidf_vect_fit):
    X_tfidf = tfidf_vect_fit.transform(data)
    words = tfidf_vect_fit.get_feature_names()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)

In [27]:
tfidf_vect = TfidfVectorizer()
#tfidf_vect_fit = tfidf_vect.fit(X_train)
X_train = tfidf_vect.fit_transform(X_train.values.astype('U'))
#X_train = vectorize(X_train,tfidf_vect_fit)

In [28]:
X_test = tfidf_vect.transform(bd_lemmatized_words['comments'].values.astype('U'))

In [29]:
X_train.shape

(11192, 15698)

In [30]:
X_test.shape

(1070, 15698)

In [31]:
y_train.shape

(11192,)

In [32]:
y_test.shape

(5596,)

# Word2Vec.

In [58]:
from gensim.models import Word2Vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [78]:
num_features = 500  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

In [79]:
print("Training model....")
model = Word2Vec(lemmatized_words,\
                workers=num_workers,\
                size=num_features,\
                min_count=min_word_count,\
                window=context,
                sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2021-01-15 22:58:08,525 : INFO : collecting all words and their counts
2021-01-15 22:58:08,526 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-15 22:58:08,548 : INFO : PROGRESS: at sentence #10000, processed 65179 words, keeping 14918 word types
2021-01-15 22:58:08,566 : INFO : PROGRESS: at sentence #20000, processed 128674 words, keeping 22840 word types
2021-01-15 22:58:08,586 : INFO : PROGRESS: at sentence #30000, processed 194264 words, keeping 29236 word types
2021-01-15 22:58:08,610 : INFO : PROGRESS: at sentence #40000, processed 258322 words, keeping 34598 word types
2021-01-15 22:58:08,634 : INFO : PROGRESS: at sentence #50000, processed 322449 words, keeping 39565 word types
2021-01-15 22:58:08,655 : INFO : PROGRESS: at sentence #60000, processed 386089 words, keeping 44137 word types
2021-01-15 22:58:08,679 : INFO : PROGRESS: at sentence #70000, processed 450981 words, keeping 48675 word types
2021-01-15 22:58:08,700 : INFO : PROGRESS: at s

Training model....


2021-01-15 22:58:08,720 : INFO : PROGRESS: at sentence #90000, processed 578376 words, keeping 56391 word types
2021-01-15 22:58:08,746 : INFO : PROGRESS: at sentence #100000, processed 643840 words, keeping 59888 word types
2021-01-15 22:58:08,767 : INFO : PROGRESS: at sentence #110000, processed 707136 words, keeping 63221 word types
2021-01-15 22:58:08,789 : INFO : PROGRESS: at sentence #120000, processed 768653 words, keeping 66362 word types
2021-01-15 22:58:08,812 : INFO : PROGRESS: at sentence #130000, processed 829112 words, keeping 69081 word types
2021-01-15 22:58:08,833 : INFO : PROGRESS: at sentence #140000, processed 890601 words, keeping 71841 word types
2021-01-15 22:58:08,859 : INFO : PROGRESS: at sentence #150000, processed 951762 words, keeping 74459 word types
2021-01-15 22:58:08,883 : INFO : PROGRESS: at sentence #160000, processed 1013405 words, keeping 76940 word types
2021-01-15 22:58:08,908 : INFO : PROGRESS: at sentence #170000, processed 1074752 words, keeping

In [80]:
model.wv.doesnt_match("москва орёл питер земля талин".split())

2021-01-15 22:58:20,358 : WARNING : vectors for words {'талин'} are not present in the model, ignoring these words
C:\Users\kamikaze666\Anaconda3\envs\tf_gpu\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'земля'

In [83]:
model.wv.most_similar("ор")

[('молча', 0.9314117431640625),
 ('батя', 0.9273248910903931),
 ('дрочить', 0.9236395955085754),
 ('стенка', 0.923109233379364),
 ('пиздить', 0.922187089920044),
 ('тереть', 0.9219539165496826),
 ('засунуть', 0.9185993671417236),
 ('палить', 0.9182090759277344),
 ('ск', 0.9169374704360962),
 ('грызть', 0.9160147905349731)]

In [77]:
model.wv.vectors.shape

(4204, 300)

In [91]:
def featureVecMethod(words, model, num_features):
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model.wv[word])

    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [92]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [93]:
trainDataVecs = getAvgFeatureVecs(lemmatized_words['comments'], model, num_features)

Review 0 of 223846
Review 1000 of 223846


<ipython-input-91-907d822aa921>:11: RuntimeWarning: invalid value encountered in true_divide
  featureVec = np.divide(featureVec, nwords)


Review 2000 of 223846
Review 3000 of 223846
Review 4000 of 223846
Review 5000 of 223846
Review 6000 of 223846
Review 7000 of 223846
Review 8000 of 223846
Review 9000 of 223846
Review 10000 of 223846
Review 11000 of 223846
Review 12000 of 223846
Review 13000 of 223846
Review 14000 of 223846
Review 15000 of 223846
Review 16000 of 223846
Review 17000 of 223846
Review 18000 of 223846
Review 19000 of 223846
Review 20000 of 223846
Review 21000 of 223846
Review 22000 of 223846
Review 23000 of 223846
Review 24000 of 223846
Review 25000 of 223846
Review 26000 of 223846
Review 27000 of 223846
Review 28000 of 223846
Review 29000 of 223846
Review 30000 of 223846
Review 31000 of 223846
Review 32000 of 223846
Review 33000 of 223846
Review 34000 of 223846
Review 35000 of 223846
Review 36000 of 223846
Review 37000 of 223846
Review 38000 of 223846
Review 39000 of 223846
Review 40000 of 223846
Review 41000 of 223846
Review 42000 of 223846
Review 43000 of 223846
Review 44000 of 223846
Review 45000 of 223

In [95]:
testDataVecs = getAvgFeatureVecs(bd_lemmatized_words['comments'], model, num_features)

Review 0 of 1070


<ipython-input-91-907d822aa921>:11: RuntimeWarning: invalid value encountered in true_divide
  featureVec = np.divide(featureVec, nwords)


Review 1000 of 1070


In [105]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
    
print("Fitting random forest to training data....")    
forest = forest.fit(trainDataVecs, lemmatized_words["sentiment"])

Fitting random forest to training data....


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [106]:
lemmatized_words["sentiment"]

0         1
1         1
2         1
3         1
4         1
         ..
223841    0
223842    0
223843    0
223844    0
223845    0
Name: sentiment, Length: 223846, dtype: int64

In [ ]:
# Predicting the sentiment values for test data and saving the results in a csv file 
result = forest.predict(testDataVecs)
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("output.csv", index=False, quoting=3 )

- Вывод: ну вы понели... код то из 2016 года!

# Doc2Vec.

In [57]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Подготовка лемматизированных данных для обучения, валидации и тестирования.

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(lemmatized_words['comments'],lemmatized_words['sentiment'],test_size = 0.90, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size = 0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size = 0.5, random_state=42)

print("Data distribution:\n- Train: {} \n- Validation: {} \n- Test: {}".format(len(y_train),len(y_val),len(y_test)))

Data distribution:
- Train: 11192 
- Validation: 5596 
- Test: 5596


In [37]:
default_list = []
for string in bd_lemmatized_words:
    new_string = ' '.join(string)
    default_list.append(new_string)

In [38]:
bd_lemmatized_words = pd.DataFrame(list(default_list), columns =['comments']) 

In [22]:
bd_lemmatized_words['sentiment'] = np.nan

In [23]:
X_test = bd_lemmatized_words['comments']

In [19]:
y_train = lemmatized_words['sentiment']

172855    0
23908     1
172208    0
162818    0
33933     1
Name: sentiment, dtype: int64

In [24]:
X_test.head()

0                               толсто аж тонко весело
1                    казаться это пародия мойти пайтон
2    типичный осада железный войнов крепость имперс...
3              просто немного унижение железный плакса
4                                           удивлённый
Name: comments, dtype: object

In [18]:
X_train.head()

172855    завтра ужасно пойти школа пойти ладный выспать...
23908            выражение услышать паршивый овца бочка мёд
172208                        задержка вылет час нафиг жить
162818                                бля вообще лень учить
33933                            помнить лето рука валанчик
Name: comments, dtype: object

In [20]:
y_test.head()

110494    1
143489    0
174023    0
120179    0
165229    0
Name: sentiment, dtype: int64

# Random Forest Classifier.

- Мы исследуем первые результаты классификатора Random Forest без какой-либо настройки гиперпараметров с использованием 5-кратной проверки.

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

RFC_clf = RandomForestClassifier(verbose=2)
scores = cross_val_score(RFC_clf, X_train, y_train.values.ravel(), cv=5)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished


In [43]:
print(scores)
scores.mean()

[0.63912461 0.63867798 0.64164433 0.62689902 0.62555853]


0.6343808934235531

In [203]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [45]:
from sklearn.model_selection import GridSearchCV

RF_clf = RandomForestClassifier(verbose=2)
parameters = {
    'n_estimators': [5,50,100],
    'max_depth': [2,10,20,None]
}

cv = GridSearchCV(RF_clf, parameters)
cv.fit(X_train,y_train.values.ravel())
print_results(cv)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


building tree 1 of 5
building tree 2 of 5
building tree 3 of 5
building tree 4 of 5
building tree 5 of 5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


building tree 1 of 5
building tree 2 of 5
building tree 3 of 5
building tree 4 of 5
building tree 5 of 5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  7.4min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.8s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 11.3min finished


# Decision Tree Classifier.

In [4]:
from sklearn import tree

In [70]:
DTC_clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=200)

In [71]:
DTC_clf.fit(X_train, y_train.values.ravel())

DecisionTreeClassifier(criterion='entropy', max_depth=200)

In [72]:
score = DTC_clf.score(X_train, y_train.values.ravel())

In [73]:
score

0.7706397426733381

In [74]:
pred = DTC_clf.predict(X_test)

In [76]:
y_test = pd.DataFrame(pred, columns = ['sentiment'])

In [77]:
y_test.to_csv('y_test.csv', index=False, encoding='cp1251')

In [ ]:
result = pd.concat([bd_lemmatized_words, df], axis=1)